## **Testando integração com a API do Wisecare**

- formatar o json pra prescription. 
- pydantic class: Org Info e AppointmentTookPlaceIn, LLM structure output

In [13]:
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
from pydantic import BaseModel, Field
from typing import List, Dict
from dotenv import load_dotenv
import os

# LOCAL IMPORTS
from agent import Doctor, AppointmentTookPlaceIn, LLMPrescription, Prescription
from agent import GENERATE_PRESCRIPTION_PROMPT
load_dotenv()  # Load environment variables from the .env file
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGSMITH_TRACING'] = os.getenv('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = os.getenv('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = os.getenv('LANGSMITH_PROJECT')

In [2]:
model = ChatOpenAI(model = "gpt-4o-mini", temperature = 0)
model = model.with_structured_output(LLMPrescription)


**Exemplo 1**

In [3]:
user_input = """Crie uma receita de Vonau de 8mg pra Roberta, genero feminino, 24 anos. Para tomar 3 vezes ao dia"""

prescription_prompt =  GENERATE_PRESCRIPTION_PROMPT.format(input = user_input , conversation_history = "")
prescription_prompt

'Você é responsável por gerar um output estruturado com base no input do médico (usuário), sem adicionar ou inventar nenhum medicamento que não tenha sido explicitamente informado.\n\nINPUT DO USUÁRIO:\nCrie uma receita de Vonau de 8mg pra Roberta, genero feminino, 24 anos. Para tomar 3 vezes ao dia\n\nHISTÓRICO DA CONVERSA (se disponível):\n\n\nINSTRUÇÕES PARA GERAÇÃO DA PRESCRIÇÃO:\n\n1. DADOS DO PACIENTE (consultant):\n   - name: Nome completo do paciente (string)\n   - age: Idade do paciente (string)\n   - gender: Gênero do paciente (string)\n\n2. MEDICAMENTOS (prescriptions):\n   Para cada medicamento que o médico indicar, forneça:\n   - name: Nome do medicamento (utilize o nome genérico, se aplicável, conforme informado)\n   - dosage: Dosagem específica (ex: "500mg", "10ml")\n   - posology: Posologia detalhada (ex: "1 comprimido a cada 8 horas", "2 gotas em cada narina 3x/dia")\n\nREGRAS IMPORTANTES:\n1. Considere o histórico completo da conversa para contextualizar a prescrição,

In [4]:
prescription_response1 = model.invoke([SystemMessage(content= prescription_prompt)])

In [5]:
json_prescription_1 = prescription_response1.dict()
json_prescription_1

{'consultant': {'name': 'Roberta', 'age': '24', 'gender': 'feminino'},
 'prescriptions': [{'name': 'Vonau',
   'dosage': '8mg',
   'posology': '1 comprimido 3 vezes ao dia'}]}

**Exemplo 2**

In [28]:
user_input = """Receita de Wellbutrin 20mg, tomar depois do cafe da manha, e Dipirona 10mg duas vezes ao dia. Paciente Kleber genero masculino, 30 anos."""

prescription_prompt2 =  GENERATE_PRESCRIPTION_PROMPT.format(input = user_input , conversation_history = "")
prescription_prompt2

'Você é responsável por gerar um output estruturado com base no input do médico (usuário), sem adicionar ou inventar nenhum medicamento que não tenha sido explicitamente informado.\n\nINPUT DO USUÁRIO:\nReceita de Wellbutrin 20mg, tomar depois do cafe da manha, e Dipirona 10mg duas vezes ao dia. Paciente Kleber genero masculino, 30 anos.\n\nHISTÓRICO DA CONVERSA (se disponível):\n\n\nINSTRUÇÕES PARA GERAÇÃO DA PRESCRIÇÃO:\n\n1. DADOS DO PACIENTE (consultant):\n   - name: Nome completo do paciente (string)\n   - age: Idade do paciente (string)\n   - gender: Gênero do paciente (string)\n\n2. MEDICAMENTOS (prescriptions):\n   Para cada medicamento que o médico indicar, forneça:\n   - name: Nome do medicamento (utilize o nome genérico, se aplicável, conforme informado)\n   - dosage: Dosagem específica (ex: "500mg", "10ml")\n   - posology: Posologia detalhada (ex: "1 comprimido a cada 8 horas", "2 gotas em cada narina 3x/dia")\n\nREGRAS IMPORTANTES:\n1. Considere o histórico completo da con

In [29]:
prescription_response2 = model.invoke([SystemMessage(content= prescription_prompt2)])
json_prescription_2 = prescription_response2.dict()
json_prescription_2

{'consultant': {'name': 'Kleber', 'age': '30', 'gender': 'masculino'},
 'prescriptions': [{'name': 'Wellbutrin',
   'dosage': '20mg',
   'posology': 'tomar depois do café da manhã'},
  {'name': 'Dipirona', 'dosage': '10mg', 'posology': 'duas vezes ao dia'}]}

In [30]:
prescription_response2

LLMPrescription(consultant=LLMPatient(name='Kleber', age='30', gender='masculino'), prescriptions=[PrescriptionItem(name='Wellbutrin', dosage='20mg', posology='tomar depois do café da manhã'), PrescriptionItem(name='Dipirona', dosage='10mg', posology='duas vezes ao dia')])

In [31]:
json_prescription_2.get('consultant')
json_prescription_2.get('prescriptions')

[{'name': 'Wellbutrin',
  'dosage': '20mg',
  'posology': 'tomar depois do café da manhã'},
 {'name': 'Dipirona', 'dosage': '10mg', 'posology': 'duas vezes ao dia'}]

## **MONTAR O PAYLOAD PRA API DA WISECARE**

In [32]:
class Prescription(BaseModel):
      """Classe Pydantic para gerar o json a ser enviado para a API do Wisecare."""
      codigo: str = '12314' # código gerado pelo Intellidoctor ou fornecido pelo Wisecare

      consultant: Dict[str,str]
      
      prescriptions: List[Dict[str, str]]  # Detalhes gerados pelo LLM - agora aceita uma lista de dicionários

      doctor: Dict[str, str]

      appointmentTookPlaceIn: Dict[str, str]

In [33]:
prescription_instance = Prescription(
    consultant = json_prescription_2.get('consultant'),
    prescriptions = json_prescription_2.get('prescriptions'),
    doctor = Doctor().model_dump(),
    appointmentTookPlaceIn = AppointmentTookPlaceIn().model_dump(),
)
prescription_instance

Prescription(codigo='12314', consultant={'name': 'Kleber', 'age': '30', 'gender': 'masculino'}, prescriptions=[{'name': 'Wellbutrin', 'dosage': '20mg', 'posology': 'tomar depois do café da manhã'}, {'name': 'Dipirona', 'dosage': '10mg', 'posology': 'duas vezes ao dia'}], doctor={'name': 'Médico Silva', 'crm': '1231231', 'uf': 'PB'}, appointmentTookPlaceIn={'name': 'Nome', 'address': 'Rua das Flores, 137', 'neighbourhood': 'Bairro', 'city': 'Cidade', 'uf': 'PB', 'phone': '11999999999'})

In [34]:
prescription_payload = prescription_instance.model_dump()
prescription_payload

{'codigo': '12314',
 'consultant': {'name': 'Kleber', 'age': '30', 'gender': 'masculino'},
 'prescriptions': [{'name': 'Wellbutrin',
   'dosage': '20mg',
   'posology': 'tomar depois do café da manhã'},
  {'name': 'Dipirona', 'dosage': '10mg', 'posology': 'duas vezes ao dia'}],
 'doctor': {'name': 'Médico Silva', 'crm': '1231231', 'uf': 'PB'},
 'appointmentTookPlaceIn': {'name': 'Nome',
  'address': 'Rua das Flores, 137',
  'neighbourhood': 'Bairro',
  'city': 'Cidade',
  'uf': 'PB',
  'phone': '11999999999'}}